In [1]:
import os
from ensure import ensure_annotations
from dataclasses import dataclass
from mlAusCar import custom_logger
from urllib import request
from pathlib import Path
from mlAusCar.constants import *
from mlAusCar.utils.common import read_yaml, create_dirs, get_size, extract_zip

In [2]:
%pwd

'd:\\My DL Workstation\\Projects\\Australian Vehicle Prices\\research'

In [3]:
os.chdir('../')
%pwd

'd:\\My DL Workstation\\Projects\\Australian Vehicle Prices'

In [4]:
# prepare entity
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path

In [5]:
# prepare configuration manager
class ConfigurationManager:
    def __init__(self,
                config_file = CONFIG_FILE_PATH,
                params_file = PARAMS_FILE_PATH,
                schema_file = SCHEMA_FILE_PATH):
    
        self.config = read_yaml(config_file)
        self.params = read_yaml(params_file)
        self.schema = read_yaml(schema_file)
        
        create_dirs([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_dirs(config.root_dir)
        
        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file
        )
        return data_ingestion_config

In [8]:
# prepare component
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            custom_logger.info(f"{filename} downloaded")
        else:
            custom_logger.info(f"{self.config.local_data_file} already exists.")
            
    def unzip_file(self):
        extract_zip(zip_file_path= self.config.local_data_file, destination=self.config.root_dir)

In [14]:
# update pipeline
try:
    config= ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_file()
except Exception as e:
    raise e

[2023-12-25 06:50:12,885]: INFO: common: .yaml from config\config.yaml has been loaded.
[2023-12-25 06:50:12,886]: INFO: common: .yaml from params.yaml has been loaded.
[2023-12-25 06:50:12,887]: INFO: common: .yaml from schema.yaml has been loaded.
[2023-12-25 06:50:12,888]: INFO: 3397765946: artifacts/data_ingestion/data.zip already exists.
